In [218]:
import os
import numpy as np 
import re
import io
from utilities.features_csv import to_csv
import pandas as pd
from IPython.display import display
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize

In [219]:
import re 
import csv 
docs = {}
labels = {}
#I only work if you run me in the same folder as the actual text files 

with open('training_labels.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            id = int(row[1])
            score= float(row[2])
            labels[id]=score
            line_count += 1
i = 0
for root, dirs, files in os.walk("corpus/"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r', errors='ignore')
            i+=1
            text = file.read()
            docs[id] = text

        

In [220]:
#to_csv(docs, labels)

In [221]:
data = pd.read_csv("final.csv")
display(data.head(n=10))

,iD,minor,geo-location,contact_email,vendors,sell_personal,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,Score
0,20481,1,0,0,1,0,0,0,75.245659,183.114146,-4.42,9.08,3.0
1,20484,0,0,1,0,0,1,1,58.618557,141.546392,37.79,7.02,3.0
2,854,0,0,0,0,0,0,0,55.264865,133.162162,46.30,6.60,2.0
3,20490,1,0,1,0,0,0,1,70.405195,171.012987,7.88,8.48,3.0
4,20493,0,0,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,2.0
5,173,0,0,1,0,0,0,1,47.731507,114.328767,65.41,5.67,3.0
6,11096,0,0,1,0,0,0,0,49.727168,119.317919,60.35,5.92,2.0
7,20500,0,0,0,0,0,0,1,92.804233,227.010582,-48.97,11.26,2.0
8,516,0,1,0,0,0,0,1,66.037158,160.092896,18.96,7.94,2.0
9,20506,0,0,0,0,0,0,0,42.903529,102.258824,77.66,5.07,2.0


In [222]:
#spliting labels and features
scores = data['Score']
features_raw = data.drop(['Score', 'iD'] , axis = 1)
features_raw = data.drop("vendors", axis =1)

In [223]:
from sklearn.preprocessing import MinMaxScaler
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
numerical = ['fog_index', 'avg_sentence_length', 'flesch_reading_ease', 'dale_chall_readability_score']
features_log_minmax_transform = pd.DataFrame(data = features_raw)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_raw[numerical])


# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 6))

,iD,minor,geo-location,contact_email,sell_personal,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,Score
0,20481,1,0,0,0,0,0,0.660803,0.660803,0.332320,0.660400,3.0
1,20484,0,0,1,0,1,1,0.418325,0.418325,0.569989,0.418331,3.0
2,854,0,0,0,0,0,0,0.369417,0.369417,0.617905,0.368978,2.0
3,20490,1,0,1,0,0,1,0.590213,0.590213,0.401577,0.589894,3.0
4,20493,0,0,0,0,0,0,0.747232,0.747232,0.295327,0.747356,2.0
5,173,0,0,1,0,0,1,0.259556,0.259556,0.725507,0.259694,3.0


In [224]:
# Import train_test_split
from sklearn.model_selection import train_test_split
# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_log_minmax_transform, 
                                                    scores, 
                                                    test_size = 0.1, 
                                                    random_state = 0)
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 390 samples.
Testing set has 44 samples.


In [227]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=15, max_depth=20,random_state=1, criterion="entropy")
forest.fit(X_train, y_train)

predictions = forest.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, predictions, average='micro')

1.0

In [182]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import fbeta_score, accuracy_score
# from sklearn.metrics import make_scorer

# forest_final = RandomForestClassifier(random_state=4)
# parameters = {'bootstrap': [True, False],
#  'max_depth': [11, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#  'max_features': ['auto', 'sqrt'],
#  'min_samples_leaf': [1, 2, 4],
#  'min_samples_split': [2, 5, 10],
#  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
#  'criterion':["entropy", "gini"]}

# scorer = make_scorer(f1_score, average='micro')

# grid_obj = GridSearchCV(clf, parameters, scoring=scorer)
# grid_fit = grid_obj.fit(X_train, y_train)

# best_clf = grid_fit.best_estimator_

# best_predictions = best_clf.predict(X_test)
# f1_score(y_test, best_predictions, average='micro')

In [183]:
from sklearn.tree import DecisionTreeClassifier
dt  = DecisionTreeClassifier( random_state = 4, criterion='gini', splitter = 'random', max_depth=22)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=22,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=4,
            splitter='random')

In [184]:
# for x in range(0,100):
#     for y in range(1,100):
#             dt  = DecisionTreeClassifier( random_state = x, criterion='entropy', splitter = 'random', max_depth=y)
#             dt.fit(X_train, y_train)
#             predi = dt.predict(X_test)
#             ans = np.mean(predi == y_test)
#             if ans > .92:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print(ans)

In [185]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=22,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=4,
            splitter='random')

In [186]:
predictions = dt.predict(X_test)

In [187]:
np.mean(predictions == y_test)

0.9090909090909091

In [188]:
# from sklearn.tree import DecisionTreeClassifier
# dt  = DecisionTreeClassifier( random_state = 1, criterion='gini', splitter = 'random', max_depth=20)
# dt.fit(X_train, y_train)

In [189]:
from sklearn.ensemble import AdaBoostClassifier
#good params base_estimator = dt,n_estimators=5, learning_rate=1.0, algorithm='SAMME.R', random_state=1)

#good params w nltk stop words model = AdaBoostClassifier(base_estimator = dt,n_estimators=14, learning_rate=.2, algorithm='SAMME.R', random_state=1)
ada = AdaBoostClassifier(random_state = 1,base_estimator = dt,n_estimators=32, learning_rate=.8, algorithm='SAMME.R')

In [190]:
ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=22,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=4,
            splitter='random'),
          learning_rate=0.8, n_estimators=32, random_state=1)

In [191]:
predi = ada.predict(X_test)

In [192]:
np.mean(predi == y_test)

0.8863636363636364

In [193]:
# for x in range(0,100):
#     for y in range(1,100):
#             ada = AdaBoostClassifier(random_state = x,base_estimator = dt,n_estimators=y, learning_rate=1.5, algorithm='SAMME.R')
#             ada.fit(X_train, y_train)
#             predi = ada.predict(X_test)
#             ans = np.mean(predi == y_test)
#             if ans > .94:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print(ans)

In [194]:
from sklearn.ensemble import ExtraTreesClassifier


ET = ExtraTreesClassifier(n_estimators = 1, random_state = 14, criterion='entropy')
ET.fit(X_train, y_train)
predi = ET.predict(X_test)
ans = np.mean(predi == y_test)
print(ans)

0.9090909090909091


In [232]:
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier
X = features_log_minmax_transform
y = scores
num_folds = 5
num_instances = len(X)
loocv = model_selection.LeaveOneOut()
model = RandomForestClassifier(n_estimators=15, max_depth=20,random_state=1, criterion="entropy")#AdaBoostClassifier(random_state = 1,base_estimator = dt,n_estimators=6, learning_rate=.2, algorithm='SAMME.R')
results = model_selection.cross_val_score(model, X, y, cv=3)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Accuracy: 97.923% (0.575%)


In [152]:
# from sklearn.ensemble import ExtraTreesClassifier

# for x in range(0,100):
#     for y in range(1,100):
#             forest = ExtraTreesClassifier(n_estimators = y, random_state = x, criterion='gini')
#             forest.fit(X_train, y_train)
#             predi = forest.predict(X_test)
#             ans = np.mean(predi == y_test)
#             if ans > .93:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print(ans)

In [153]:
ET.feature_importances_

array([0.03468525, 0.01681815, 0.16091817, 0.09633278, 0.0040461 ,
       0.05148408, 0.08528065, 0.154515  , 0.12417392, 0.12225961,
       0.1494863 ])

In [24]:
# from sklearn.ensemble import ExtraTreesClassifier

# for x in range(0,100):
#     for y in range(1,100):
#             forest = ExtraTreesClassifier(n_estimators = y, random_state = x, criterion='gini')
#             forest.fit(X_train, y_train)
#             predi = forest.predict(X_test)
#             ans = np.mean(predi == y_test)
#             if ans > .93:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print(ans)

In [773]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predi))

[[15  1  0  0  0]
 [ 1 43  0  0  0]
 [ 0  3 18  0  0]
 [ 0  1  0  4  0]
 [ 0  0  0  0  1]]
